In [2]:
import whisper

In [3]:
model = whisper.load_model("large-v3")

In [4]:
TESTF = "/home/joregan/Listening_test/00_4.wav"

In [5]:
audio = whisper.load_audio(TESTF)
audio = whisper.pad_or_trim(audio)

In [6]:
mel = whisper.log_mel_spectrogram(audio, n_mels=128).to(model.device)

In [7]:
mel.shape

torch.Size([128, 3000])

In [9]:
res = model.transcribe(TESTF, verbose=True)

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Swedish
[00:00.000 --> 00:03.100]  Tror du att jag är engelsman eller amerikan?


In [10]:
res

{'text': ' Tror du att jag är engelsman eller amerikan?',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.1,
   'text': ' Tror du att jag är engelsman eller amerikan?',
   'tokens': [50365,
    1765,
    284,
    1581,
    951,
    6368,
    3775,
    1741,
    1625,
    1601,
    12519,
    16116,
    16048,
    30,
    50520],
   'temperature': 0.0,
   'avg_logprob': -0.04304603487253189,
   'compression_ratio': 0.8490566037735849,
   'no_speech_prob': 0.1451009064912796}],
 'language': 'sv'}

In [13]:
len(res["segments"][0]["tokens"])

15

In [8]:
language_tokens, probs = model.detect_language(mel)

In [43]:
probs

{'ka': 1.1760527485193961e-07,
 'uk': 0.00011050760076614097,
 'ps': 4.126469832499424e-07,
 'ba': 4.256304642069608e-09,
 'af': 1.924401431097067e-06,
 'kn': 5.272009957479895e-07,
 'so': 5.345439646475825e-09,
 'ne': 5.848129376317956e-07,
 'ja': 0.0008843151153996587,
 'mk': 3.5319104085829167e-07,
 'lo': 4.839035909753875e-07,
 'hi': 0.00012629431148525327,
 'zh': 0.0011262395419180393,
 'lv': 1.6683612557244487e-05,
 'et': 1.52700231410563e-05,
 'az': 8.513211469107773e-06,
 'sd': 7.044382641652192e-07,
 'ml': 0.00019973318558186293,
 'no': 0.02231231890618801,
 'is': 0.00010205624130321667,
 'ln': 9.586413796114357e-08,
 'yi': 2.436002887407085e-06,
 'ar': 0.0004886251408606768,
 'ur': 1.1246702342759818e-05,
 'tl': 0.0005676936125382781,
 'bn': 3.0907585824024864e-06,
 'am': 6.384588857599738e-08,
 'mt': 1.5055169342303998e-06,
 'yo': 6.955768867555889e-07,
 'id': 0.00020396412583068013,
 'tt': 8.788865635267484e-09,
 'la': 0.0005797904450446367,
 'mr': 6.714851110700693e-07,
 '

In [46]:
print("Swedish", probs['sv'], "English", probs["en"])

Swedish 0.9029163122177124 English 0.038634639233350754


In [19]:
from whisper.decoding import detect_language

In [40]:
from whisper.tokenizer import LANGUAGES

In [42]:
len(LANGUAGES)

100

In [ ]:
detect_language("Whisper", mel=mel)